### Selected Word Group In Youtube Sentence

In [1]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [2]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "Arabic"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# native word select for Part 2 
word_start = 0  # 0  # native word start index
word_end = 200 # 28  # native word end index

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish"

# parameter
file = "Turkish_28_Word_Threegram_Manuel.xlsx"  # without xlsx extention
sheets = "Sheet1"  # 2 gram target, 2 gram hybrid
file_ext = "28_Word_Threegram"
first_column = 0  # start column location number
end_column = 1
time_shift = 0.6  # 0.6
sample_num = 6  # 6

# youtube sentence condition
sent_time_start = False  # True, False for activating start time condition
sent_start_time = 700  # sn
random_sample = True  # True, False

In [3]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/7-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [4]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [5]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [6]:
def word_count_result(df, column_list, set_condition=False): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        if set_condition:
            var_list = set(df[f"{i}"].dropna().tolist())
        else:
            var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    df_word_count.sort_values("word_count", ascending=False, inplace=True)
    df_word_count.reset_index(inplace=True, drop=True)
    
    return  df_word_count

In [7]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
987924,karneleme,5
987925,karnaya,5
987926,dörtlümüzün,5
987927,karnavalınız,5


In [8]:
df_word_select = df_word_all.iloc[word_start:word_end,]
df_word_select

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
195,saat,399989
196,onunla,399330
197,yapıyorsun,398274
198,neler,397377


In [9]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:00.294,00:00:03.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,00:00:03.375,00:00:06.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,00:00:06.462,00:00:09.462,wwwsebederorg,fI1BI4d5KFU
3,00:00:10.267,00:00:11.394,mumu,fI1BI4d5KFU
4,00:00:11.835,00:00:14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,00:10:00.240,00:10:07.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,00:10:07.994,00:10:13.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,00:10:14.745,00:10:17.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,00:10:18.047,00:10:22.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [10]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [11]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,0.294,3.294,bu dizinin betimlemesi staff fi lm tarafından,fI1BI4d5KFU
1,3.375,6.375,sesli betimleme derneğine yaptırılmıştır,fI1BI4d5KFU
2,6.462,9.462,wwwsebederorg,fI1BI4d5KFU
3,10.267,11.394,mumu,fI1BI4d5KFU
4,11.835,14.061,bak şu an sinir katsayım hat safhada,fI1BI4d5KFU
...,...,...,...,...
3934202,600.240,607.818,videoyu bitirmeden şunu da söylemek isterim ki...,dRssR_apbR0
3934203,607.994,613.534,bu nedenle ulaşımda insanların yarısına yakını...,dRssR_apbR0
3934204,614.745,617.924,danimarkayla alakalı şimdilik söyleyeceklerim ...,dRssR_apbR0
3934205,618.047,622.398,video hoşunuza gittiyse beğenmeyi ve kanalıma ...,dRssR_apbR0


In [12]:
if sent_time_start:
    df_youtube_select = df_youtube_sentence[df_youtube_sentence["start_time"] <= sent_start_time]
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    df_youtube_select = df_youtube_sentence

if random_sample:
    df_youtube_select = df_youtube_select.sample(len(df_youtube_select))
    df_youtube_select.reset_index(drop=True, inplace=True)
else:
    pass

df_youtube_select

,start_time,end_time,sentence,video_id
0,604.231,606.219,böyle bak oraya bak falan,XREYOwgg6Uk
1,2291.756,2294.853,yok canım merak ediyorum,EPA02wTaonI
2,910.500,911.514,ne oldu,98xXlNVvfq4
3,33.000,39.171,üzerine,arltOsA2Iwk
4,161.038,163.604,istediğiniz gibi istediğiniz zaman da değiştir...,B_DJOqGf8Lw
...,...,...,...,...
3934202,206.460,214.470,tv,porSPRja_EA
3934203,2767.391,2768.838,tamam ben konuşurum metin müdürle,K4oWnQPSF10
3934204,6306.075,6309.647,melisanın ailesi kalabalık çıkınca benim iş zo...,s6vN0Sl2F1c
3934205,1663.530,1666.860,şurada eğitim başlıyoooorrrr diyeyim,xiaSoh6_9MI


In [13]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,threegram
0,biz de öyle
1,biz de seni
2,yok artık daha
3,yok artık ya
4,gerek yok artık
...,...
199,oh evet efendim
200,oh evet ben
201,ne zaman oldu
202,hayır efendim ben


In [ ]:
# Option 
# Skip if it doesn't need
df_remove = pd.read_excel(f"")
df_remove

In [ ]:
remove_videoid_list = list(set(df_remove["video_id"]))
#remove_videoid_list

In [ ]:
len(remove_videoid_list)

In [ ]:
# Option
df_youtube_select = df_youtube_select[~df_youtube_select["video_id"].isin(remove_videoid_list)]
df_youtube_select

In [14]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[first_column:end_column]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
            search_list = list(set(search_list))  # for unique value
        else:
            pass

In [15]:
len(search_list)

200

In [16]:
df_search_result = word_group_youtube(df_youtube_select, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,hayır doğru değil,8080.408,8081.458,hayır doğru değil,4RMoZo69RE8
1,hayır doğru değil,993.408,995.208,hayır doğru değil,Isab2NO9jlk
2,hayır doğru değil,5833.306,5834.508,yok hayır doğru değil,Ik61qAqcnXo
3,hayır doğru değil,33.478,34.918,hayır doğru değil,PeQifrSZmBo
4,peki ya sen,387.915,389.112,peki ya sen,Qs1I7HxXYlg
...,...,...,...,...,...
929,belki daha sonra,5983.247,5985.825,belki daha sonra oynarsınız olur mu,g81mNCrBnO4
930,belki daha sonra,2261.644,2264.338,canım belki daha sonra bahçeye bir şey yapacaktır,WLGk7o3R5Es
931,belki daha sonra,421.596,424.284,canım belki daha sonra bahçeye bir şey yapacaktır,ccnwNHUwrP0
932,belki daha sonra,2994.012,2996.012,belki daha sonra seni bir yerlere götürürüm,X48jgyzoQdE


In [17]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hayır doğru değil,8080.408000,8081.458000,hayır doğru değil,4RMoZo69RE8
1,hayır doğru değil,993.408000,995.208000,hayır doğru değil,Isab2NO9jlk
2,hayır doğru değil,5833.477714,5834.508000,yok hayır doğru değil,Ik61qAqcnXo
3,hayır doğru değil,33.478000,34.918000,hayır doğru değil,PeQifrSZmBo
4,peki ya sen,387.915000,389.112000,peki ya sen,Qs1I7HxXYlg
...,...,...,...,...,...
929,belki daha sonra,5983.247000,5984.499171,belki daha sonra oynarsınız olur mu,g81mNCrBnO4
930,belki daha sonra,2261.918898,2262.908531,canım belki daha sonra bahçeye bir şey yapacaktır,WLGk7o3R5Es
931,belki daha sonra,421.870286,422.857714,canım belki daha sonra bahçeye bir şey yapacaktır,ccnwNHUwrP0
932,belki daha sonra,2994.012000,2994.802698,belki daha sonra seni bir yerlere götürürüm,X48jgyzoQdE


In [18]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,hayır doğru değil,8079.808000,8082.058000,hayır doğru değil,4RMoZo69RE8
1,hayır doğru değil,992.808000,995.808000,hayır doğru değil,Isab2NO9jlk
2,hayır doğru değil,5832.877714,5835.108000,yok hayır doğru değil,Ik61qAqcnXo
3,hayır doğru değil,32.878000,35.518000,hayır doğru değil,PeQifrSZmBo
4,peki ya sen,387.315000,389.712000,peki ya sen,Qs1I7HxXYlg
...,...,...,...,...,...
929,belki daha sonra,5982.647000,5985.099171,belki daha sonra oynarsınız olur mu,g81mNCrBnO4
930,belki daha sonra,2261.318898,2263.508531,canım belki daha sonra bahçeye bir şey yapacaktır,WLGk7o3R5Es
931,belki daha sonra,421.270286,423.457714,canım belki daha sonra bahçeye bir şey yapacaktır,ccnwNHUwrP0
932,belki daha sonra,2993.412000,2995.402698,belki daha sonra seni bir yerlere götürürüm,X48jgyzoQdE


In [19]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,hayır doğru değil,8080,8082,hayır doğru değil,4RMoZo69RE8
1,hayır doğru değil,993,996,hayır doğru değil,Isab2NO9jlk
2,hayır doğru değil,5833,5835,yok hayır doğru değil,Ik61qAqcnXo
3,hayır doğru değil,33,36,hayır doğru değil,PeQifrSZmBo
4,peki ya sen,387,390,peki ya sen,Qs1I7HxXYlg
...,...,...,...,...,...
929,belki daha sonra,5983,5985,belki daha sonra oynarsınız olur mu,g81mNCrBnO4
930,belki daha sonra,2261,2264,canım belki daha sonra bahçeye bir şey yapacaktır,WLGk7o3R5Es
931,belki daha sonra,421,423,canım belki daha sonra bahçeye bir şey yapacaktır,ccnwNHUwrP0
932,belki daha sonra,2993,2995,belki daha sonra seni bir yerlere götürürüm,X48jgyzoQdE


In [20]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,hayır doğru değil,8080,8082,hayır doğru değil,4RMoZo69RE8,https://www.youtube.com/watch?v=4RMoZo69RE8&t=...
1,hayır doğru değil,993,996,hayır doğru değil,Isab2NO9jlk,https://www.youtube.com/watch?v=Isab2NO9jlk&t=...
2,hayır doğru değil,5833,5835,yok hayır doğru değil,Ik61qAqcnXo,https://www.youtube.com/watch?v=Ik61qAqcnXo&t=...
3,hayır doğru değil,33,36,hayır doğru değil,PeQifrSZmBo,https://www.youtube.com/watch?v=PeQifrSZmBo&t=33s
4,peki ya sen,387,390,peki ya sen,Qs1I7HxXYlg,https://www.youtube.com/watch?v=Qs1I7HxXYlg&t=...
...,...,...,...,...,...,...
929,belki daha sonra,5983,5985,belki daha sonra oynarsınız olur mu,g81mNCrBnO4,https://www.youtube.com/watch?v=g81mNCrBnO4&t=...
930,belki daha sonra,2261,2264,canım belki daha sonra bahçeye bir şey yapacaktır,WLGk7o3R5Es,https://www.youtube.com/watch?v=WLGk7o3R5Es&t=...
931,belki daha sonra,421,423,canım belki daha sonra bahçeye bir şey yapacaktır,ccnwNHUwrP0,https://www.youtube.com/watch?v=ccnwNHUwrP0&t=...
932,belki daha sonra,2993,2995,belki daha sonra seni bir yerlere götürürüm,X48jgyzoQdE,https://www.youtube.com/watch?v=X48jgyzoQdE&t=...


In [21]:
search_result_num = df_word_group_time_loc_result["search_string"].nunique()
search_result_num

183

In [22]:
search_result_set = set(df_word_group_time_loc_result["search_string"])
search_all_set = set(search_list)

In [23]:
search_all_set.difference(search_result_set)

{'artık burada değil',
 'bana ver şunu',
 'başka sorun var',
 'evet bunu biliyorum',
 'haydi ama bana',
 'haydi ama dostum',
 'hey ver onu',
 'lütfen söyle bana',
 'neden burada değil',
 'oh evet ben',
 'oh evet doğru',
 'oh evet efendim',
 'oh evet tabii',
 'oh gerçekten mi',
 'oh hayır hayır',
 'sanırım buraya kadar',
 've bunu biliyorsun'}

In [24]:
df_result_word_manuel_count = word_count_result(df_word_group_time_loc_result, df_word_group_time_loc_result.columns[0:1], set_condition=True)
df_result_word_manuel_count

,word,word_count
0,bu,28
1,değil,24
2,mi,18
3,yok,16
4,çok,16
...,...,...
106,bilmiyorum,1
107,yüzden,1
108,biri,1
109,teşekkürler,1


In [25]:
# # Option merge
# df_word_count_merge = pd.merge(df_result_word_manuel_count, df_word_select[["word"]], how="inner", on="word")
# df_word_count_merge.drop_duplicates(inplace=True)
# df_word_count_merge.sort_values(by="word_count", ascending=False, inplace=True)
# df_word_count_merge.reset_index(drop=True, inplace=True)
# df_word_count_merge

In [26]:
set_word = set(df_word_select["word"])
#set_count = set(df_word_count_merge["word"])  # Option merge
set_count = set(df_result_word_manuel_count["word"])

In [27]:
set_word.difference(set_count)

{'adam',
 'al',
 'asla',
 'aslında',
 'baba',
 'bak',
 'bakalım',
 'bence',
 'benimle',
 'bile',
 'biliyor',
 'birkaç',
 'birlikte',
 'bize',
 'bizi',
 'bugün',
 'buna',
 'bütün',
 'büyük',
 'dakika',
 'devam',
 'diye',
 'dostum',
 'dur',
 'ederim',
 'en',
 'et',
 'eğer',
 'gel',
 'geldi',
 'geliyor',
 'gerçek',
 'git',
 'göre',
 'gün',
 'hakkında',
 'haydi',
 'her',
 'iki',
 'ile',
 'ilk',
 'ister',
 'istiyorum',
 'izin',
 'içinde',
 'iş',
 'işte',
 'kendi',
 'küçük',
 'kız',
 'lazım',
 'musun',
 'mü',
 'mısın',
 'nasıl',
 'neler',
 'ol',
 'olan',
 'olarak',
 'olduğunu',
 'olmak',
 'olsun',
 'olur',
 'ona',
 'onlar',
 'onun',
 'onunla',
 'pekala',
 'saat',
 'sanırım',
 'seninle',
 'siz',
 'size',
 'sizi',
 'son',
 'tam',
 'teşekkür',
 'tüm',
 'uzun',
 'vardı',
 've',
 'yardım',
 'yeni',
 'zaten',
 'önce',
 'özür',
 'üç',
 'şekilde',
 'şeyi'}

In [28]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [29]:
# df_result_word_manuel_count.to_excel(f"{lang_folder.capitalize()}_{search_result_num}_{file_ext}_\
# {sample_num}_Youtube_{time_shift}s_Timeshift_Result_Word_Count.xlsx", index=False) 

#### Copy Move And Delete

In [30]:
output_file = glob.glob(f"{lang_folder.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result*.xlsx")
output_file

['Turkish_183_Threegram_With_28_And_172_Word_Pair_Selected_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [31]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [32]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass

#### Temp

In [ ]:
set_result = set(df_word_group_time_loc_result["search_string"].unique())
set_selected = set(search_list)

In [ ]:
df_not_in_result = pd.DataFrame(set_selected.difference(set_result), columns=["not_in_youtube_result"])
df_not_in_result

In [ ]:
not_in_result_num = len(df_not_in_result)
not_in_result_num

In [ ]:
#df_not_in_result.to_excel(f"{lang_folder.capitalize()}_{not_in_result_num}_{file_ext}_\
#Not_In_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

In [ ]:
search_list2 = df_not_in_result["not_in_youtube_result"].to_list()

In [ ]:
df_search_result2 = word_group_youtube(df_youtube_sentence, search_list2, "sentence", sample_num)
df_search_result2

In [ ]:
search_result2_num = df_search_result2["search_string"].nunique()
search_result2_num

In [ ]:
df_word_group_time_loc_result2 = word_group_time_loc(df_search_result2, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2.start_time = df_word_group_time_loc_result2.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result2.end_time = df_word_group_time_loc_result2.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result2

In [ ]:
df_word_group_time_loc_result2["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result2['video_id'].map(str)+"&t="+df_word_group_time_loc_result2['start_time'].map(str)+"s"
df_word_group_time_loc_result2

In [ ]:
search_result_num2 = df_word_group_time_loc_result2["search_string"].nunique()
search_result_num2

In [ ]:
df_word_group_time_loc_result2.to_excel(f"{lang_folder.capitalize()}_{search_result_num2}_{file_ext}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result2.xlsx", index=False) 